## The `chatlas` package for a consistent LLM interface and workflow

One of the frustrations of working with different LLM providers is the difference in API structures.  This has historically meant that developers have had to code up their LLM workflows quite differently if they were working with, say, OpenAI versus Anthropic or Llama.

The `chatlas` package attempts to overcome this by offering a set of classes and methods that have greater alignment across LLM providers.  This means that starting a chat, or using tool calling or other services will look the same or very similar, no matter which LLM provider you are using.  For those familiar, `chatlas` is basically the Python equivalent of R's `ellmer` package.

The following is a short illustratory demo of some of the features of `chatlas`.

### Starting a chat session - OpenAI example

In [3]:
# import packages and key environment variables
from dotenv import load_dotenv
from chatlas import ChatOpenAI, ChatAnthropic, ChatOllama
import os

load_dotenv()
INSTANCE_ID = os.getenv('INSTANCE_ID')
API_KEY = os.getenv('API_KEY')
BASE_URL_STEM = os.getenv('BASE_URL_STEM')

In [4]:
# set up an openai chat client
PROVIDER = "openai"
BASE_URL = f"https://{PROVIDER}.{BASE_URL_STEM}/{INSTANCE_ID}/v1"

chat = ChatOpenAI(
    model = "gpt-4o",
    api_key = API_KEY,
    base_url = BASE_URL,
    system_prompt = "You are a friendly but terse assistant.",
)

In [ ]:
# start a chat app
chat.app()

INFO:     Started server process [3200]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:39685 (Press CTRL+C to quit)


INFO:     127.0.0.1:51677 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:51678 - "GET /lib/jquery-3.6.0/jquery-3.6.0.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:51677 - "GET /lib/requirejs-2.3.6/require.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:51679 - "GET /lib/shiny-busy-indicators-1.4.0/busy-indicators.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:51681 - "GET /lib/shiny-1.4.0/shiny.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:51680 - "GET /lib/bootstrap-5.3.1/bootstrap.min.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:51682 - "GET /lib/bootstrap-5.3.1/bootstrap.bundle.min.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:51679 - "GET /lib/shiny-chat-output-1.4.0/chat.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:51680 - "GET /lib/shiny-markdown-stream-1.4.0/markdown-stream.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:51679 - "GET /lib/shiny-textarea-autoresize-1.4.0/textarea-autoresize.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:51679 - "GET /lib/htmltools-fill-0.5.8.9000/fill.css HTTP/1.1" 200 OK
INFO:     127.0.0.1

INFO:     ('127.0.0.1', 51683) - "WebSocket /websocket/" [accepted]
INFO:     connection open


INFO:     127.0.0.1:51680 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     connection closed


In [ ]:
# chat in the console
chat.console()

In [6]:
# programmatic chat
chat.chat("What exactly is a spirit vegetable?")

<br>

A spirit vegetable is more of a whimsical or humorous concept rather than something with a strict definition. It’s often used in a lighthearted way to describe a vegetable that resonates with someone’s personality or that they feel a special connection to. Think of it like a “spirit animal,” but in the vegetable world!

### Tool (function) calling - OpenAI example

In [7]:
# tool (function) calling - function to get current temperature
import requests

# function to get the current temperature in a place
def get_current_temperature(place: str) -> str:
  """Get the current temperature in a given place."""
  base_url = f"https://wttr.in/{place}?format=j1"
  response = requests.get(base_url)
  data = response.json()
  return f"The current temperature in {place} is {data['current_condition'][0]['temp_C']} degrees Celsius"

# test the function
get_current_temperature("London")

'The current temperature in London is 13 degrees Celsius'

In [8]:
# register the function with your chat
chat.register_tool(get_current_temperature)

In [9]:
# now the chat can use it
chat.chat("I'm in Atlanta today and I'm told I should wear warm clothes.  What do you think?")

<br>



```python
# 🔧 tool request (call_tE5xW2RroS8scuWaCkm8tFco)
get_current_temperature(place=Atlanta)
```




```python
# ✅ tool result (call_tE5xW2RroS8scuWaCkm8tFco)
The current temperature in Atlanta is 28 degrees Celsius
```

<br>

The current temperature in Atlanta is 28°C (82°F), which is quite warm. You probably don't need warm clothes; lightweight and breathable clothing would be more comfortable.

### Structured data extraction - OpenAI example

In [10]:
# pulling structured data out of text
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    pets: int
    skills: list[str]

chat.extract_data(
  "My name is Keith.  I have two cats and one dog named Bertie.  I am very good at Math and Computer Games", 
  data_model=Person,
)

{'name': 'Keith', 'pets': 3, 'skills': ['Math', 'Computer Games']}

### Start a new chat - Anthropic example

In [11]:
# now try a Anthropic chat client- note similar but not 100% identical to OpenAI
PROVIDER = "anthropic"
BASE_URL = f"https://{PROVIDER}.{BASE_URL_STEM}/{INSTANCE_ID}"

chat = ChatAnthropic(
    api_key = API_KEY,
    system_prompt = "You are a friendly but terse assistant.",
    kwargs = {"base_url": BASE_URL}  
)

In [12]:
# programmatic chat
chat.chat("Which integer is commonly quoted as the answer to the meaning of life?")

<br>

42.

In [13]:
# register tool
chat.register_tool(get_current_temperature)

In [14]:
# check tool use
chat.chat("My sister is heading to the capital of Norway next week?  How should she pack?")

<br>

To advise on packing for your sister's trip to Oslo (the capital of Norway), I'll need to check the current temperature there.

```python
# 🔧 tool request (toolu_01NRn7QGuVXV7di9MYjGR8oj)
get_current_temperature(place=Oslo, Norway)
```




```python
# ✅ tool result (toolu_01NRn7QGuVXV7di9MYjGR8oj)
The current temperature in Oslo, Norway is 19 degrees Celsius
```

<br>

Based on the current temperature of 19°C (66°F) in Oslo, your sister should pack:

- Light to medium weight clothing
- A light jacket or sweater for cooler evenings
- Comfortable walking shoes
- A light rain jacket (Oslo can experience occasional rain)
- Layers to adjust to temperature changes throughout the day

The weather is relatively mild right now, but she should check the forecast closer to her travel date as conditions may change.

In [15]:
#  extract structured data
class Person(BaseModel):
    name: str
    pets: int
    areas_of_expertise: list[str]
    qualifications: list[str]

chat.extract_data(
  """
  My name is Keith.  I had two cats and one dog named Bertie, but I recently sold the two cats.  I have a PhD in Pure Mathematics, and I am also a holder of the Licentiate Performing Diploma from Trinity College of Music in London, taking my final exams in the Recorder.
  """, 
  data_model = Person,
)

{'name': 'Keith',
 'pets': 1,
 'areas_of_expertise': ['Pure Mathematics', 'Music', 'Recorder'],
 'qualifications': ['PhD in Pure Mathematics',
  'Licentiate Performing Diploma from Trinity College of Music in London']}

In [16]:
# update structured information during chat session
chat.extract_data(
  """
  Oh it's Keith again.  I'm sorry, I'm so dumb.  I forgot to mention I also have a new puppy which we only just picked up this week.
  """, 
  data_model = Person,
)

{'name': 'Keith',
 'pets': 2,
 'areas_of_expertise': ['Pure Mathematics', 'Music', 'Recorder'],
 'qualifications': ['PhD in Pure Mathematics',
  'Licentiate Performing Diploma from Trinity College of Music']}

In [17]:
# update again
chat.extract_data(
  """
  It's Keith again.  I just spoke to my wife and she left the door open and the puppy ran out on the street and got killed.  It's a bad day.
  """, 
  data_model = Person,
)

{'name': 'Keith',
 'pets': 1,
 'areas_of_expertise': ['Pure Mathematics', 'Music', 'Recorder'],
 'qualifications': ['PhD in Pure Mathematics',
  'Licentiate Performing Diploma from Trinity College of Music']}

### New chat session with local model using `ollama`

In [18]:
# local ollama model
chat = ChatOllama(
    model = "llama3.1:8b",
    system_prompt = "You are a friendly but terse assistant.",
)

In [19]:
# programmatic chat
chat.chat("Explain Buddhism in two sentences")

<br>

Buddhism is a spiritual path that aims to help individuals achieve enlightenment and end suffering through the understanding of the Four Noble Truths: that life is full of impermanence, dissatisfaction, and uncertainty. By cultivating mindfulness, detachment, and compassion, Buddhists strive towards a state of Nirvana, where all desires and egoic tendencies are transcended.

In [20]:
# extract data using earlier data model
chat.extract_data(
      """
  My name is Keith.  I had two cats and one dog named Bertie, but I recently sold the two cats.  I have a PhD in Pure Mathematics, and I am also a holder of the Licentiate Performing Diploma from Trinity College of Music in London, taking my final exams in the Recorder.
  """, 
  data_model = Person,
)

{'name': 'Keith',
 'pets': 1,
 'areas_of_expertise': ['Pure Mathematics', 'Recorder'],
 'qualifications': ['PhD',
  'Licentiate Performing Diploma (Trinity College of Music)']}

In [21]:
# register temperature tool
chat.register_tool(get_current_temperature)

In [22]:
# use temperature tool
chat.chat("I'm heading to Tasmania tomorrow but I forgot my sunscreen.  Should I be concerned?")

<br>



```python
# 🔧 tool request (call_ri95nnxj)
get_current_temperature(place=Hobart)
```




```python
# ✅ tool result (call_ri95nnxj)
The current temperature in Hobart is 10 degrees Celsius
```

<br>

You might want to consider purchasing some sunscreen in Tasmania, as the UV index and sun protection rating are moderate to high during your visit. You can find sunscreen at most pharmacies or supermarkets in major towns like Hobart and Launceston. Don't worry too much about it, but a bit of precaution won't hurt!